In [11]:
!pip install streamlit -q

     |████████████████████████████████| 7.4MB 6.5MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 163kB 43.1MB/s 
     |████████████████████████████████| 112kB 54.8MB/s 
     |████████████████████████████████| 4.5MB 33.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 122kB 35.6MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.


In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-12-04 13:27:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.225.42.45, 3.230.235.205, 52.1.26.21, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.225.42.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  36.0MB/s    in 0.4s    

2020-12-04 13:27:58 (36.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [3]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 3.3MB/s 
     |████████████████████████████████| 133kB 10.4MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
     |████████████████████████████████| 7.0MB 16.2MB/s 
ERROR: botocore 1.19.29 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [4]:
! pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 1.1MB 25.3MB/s 
     |████████████████████████████████| 2.9MB 35.3MB/s 
     |████████████████████████████████| 890kB 36.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=da48bc389ef6d60a1da57bfabfc622530a5a3bf38257697e434a485a5384ee5e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%%writefile web.py

import pytorch_pretrained_bert
import streamlit as st
import numpy as np
import pandas as pd
import altair as alt
from pip._vendor.urllib3.packages.six import StringIO
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
from tqdm import tqdm
import time
from datetime import timedelta
from importlib import import_module
import torch
import numpy as np
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelWithLMHead, AdamW
import torch.nn.functional as F
from sklearn import metrics
from pytorch_pretrained_bert import *
import transformers

import matplotlib.pyplot as plt
import spacy   # for tokenising text
from spacy.lang.en import English  # for tokenising text
nlp = English()   # for tokenising text
from collections import Counter   # for getting freq of words
from spacy.matcher import PhraseMatcher
import seaborn as sns   # for charts
sns.set_style("whitegrid");   # chart background style
plt.rcParams['figure.dpi'] = 360   # for high res chart output
import re   # for regular expressions

PAD, CLS = '[PAD]', '[CLS]'


def main():
    st.title("Toxic Word Detection API")

    activities = ['Detection', 'Dataset Display', 'Dataset Analysis 1', 'Dataset Analysis 2']
    choice = st.sidebar.selectbox("Select Choice", activities)

    if choice == 'Detection':
        st.subheader("Toxic Word Detection")
        user_input = acp_use_input()
        st.write('The text is', user_input)

        if len(user_input) == 0:
            st.info("Please enter the text you want to classify!")
            # st.text("Please enter the text: ")
            
        else:

            print("input is ", user_input)

            test_data = load_dataset(user_input, config.pad_size)
            print("test data is ", test_data)
            test_iter = build_iterator(test_data, config)
            print('datatiter is ', test_iter.batch_size)
            print('datatiter is ', test_iter.batches)
            print('datatiter is ', test_iter.device)

            model = Model(config).to(config.device)
            model.load_state_dict(torch.load(config.save_path, map_location=torch.device('cpu')),  strict=False)
            model.eval()

            predict_label = evaluate(config, model, test_iter, test=True)

            if predict_label == 1:
              st.write("The predicted label for entered text is Toxic!")
            else:
              st.write("The predicted label for entered text is Non-toxic!")           

            # outputs = BERT_Predict(model, user_input)

    elif choice == "Dataset Display":
        st.subheader("Dataset Display")
        path_chat_data = '/content/gdrive/My Drive/materials/labeled_Chat.csv'
        df = load_data(path_chat_data)
        # df.columns = ['match_id', 'key', 'slot', 'time', 'unit', 'label']
        df.rename(columns = {'0': 'label'}, inplace=True)
        df.loc[1, 'label'] = 0
        # df = df.head(20)
        # st.dataframe(df, 1500, 1200)

        contents_filter = st.slider("Choose only to show toxic chat or non-toxic chat", 0, 2, 2, 1)
        st.info("Select 0 will show non-toxic chat; Select 1 will show toxic chat; Select 2 will show all data")
        if contents_filter == 2:
          df = df.head(20)
          st.dataframe(df, 1500, 1200)
        elif contents_filter == 1:
          df_filted = df[df['label'] == 1]
          st.dataframe(df_filted, 1500, 1200)          
        else:         
          df_filted = df[df['label'] == 0]
          df_filted = df_filted.head(20)
          st.dataframe(df_filted, 1500, 1200)

    # new feature WEEK9
    elif choice == 'Dataset Analysis 1':
        st.set_option('deprecation.showPyplotGlobalUse', False)

        if st.checkbox('1. Communication frequency of all matches', 1):
            # st.subheader('1. Communication frequency of all matches')
            path_chat_data = '/content/gdrive/My Drive/materials/chat.csv'
            df_chat_data = load_data(path_chat_data)

            time = df_chat_data['time']
            # fig1 = plt.subplots()
            plt.figure(1)
            plt.hist(time, bins=100, facecolor="purple")
            plt.ylabel("Count of chat")
            plt.xlabel("Time series(second)")
            plt.title("Communication frequency of all matches")
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that the communication in the game mainly concentrated on around 2500 sec.
                """)
            # st.set_option('deprecation.showPyplotGlobalUse', False)

        # if st.checkbox('2. Player win rate distribution', 2):
        #     # st.subheader('')
        #     # draw_figure2()
        #     path_data = '/content/gdrive/My Drive/materials/player_ratings.csv'
        #     df_data = load_data(path_data)
        #     total_wins = df_data["total_wins"].values
        #     total_matches = df_data["total_matches"].values


        #     total_wins, total_matches = (list(t) for t in zip(*sorted(zip(total_wins, total_matches))))
        #     plt.figure(2)
        #     plt.plot(total_wins[0:-1], total_matches[0:-1])
        #     plt.xlabel('total matches')
        #     plt.ylabel('total wins')

        #     st.pyplot()

        #     with st.beta_expander("Check analysis"):
        #         st.write("""
        #             This figure indicates that the win rate of most players is approximately 50%.
        #         """)

        if st.checkbox('2. Player win rate distribution', 2):
            path_data = '/content/gdrive/My Drive/materials/player_ratings.csv'
            df_data = load_data(path_data)
            total_wins = df_data["total_wins"].values
            total_matches = df_data["total_matches"].values


            total_wins, total_matches = (list(t) for t in zip(*sorted(zip(total_wins, total_matches))))
            y=total_wins[0:-1]
            x=total_matches[0:-1]        

            plt.figure(2)

            plt.scatter(x,y)
            z= np.polyfit(x, y, 1)
            p = np.poly1d(z)

            plt.plot(x,p(x),"-",color='#FFA500')
            plt.xlabel('total matches ')
            plt.ylabel('total wins')
            plt.title('Player win rates distribution')
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that the player's true skill will tend to be stable while the total matches increases.
                """)

        if st.checkbox('3. Player skills level distribution (mu)', 3):
            # st.subheader('')
            path_data = '/content/gdrive/My Drive/materials/player_ratings.csv'
            df_data = load_data(path_data)
            trueskill_mu1 = df_data["trueskill_mu"].values

            plt.figure(3)

            plt.hist(trueskill_mu1, bins=40, histtype='bar', edgecolor="black", align="mid")
            plt.xlabel('trueskill mu')
            plt.ylabel("Players")
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that overall skill of players follows the mu curve.
                """)


        if st.checkbox('4. Relation between kills and toxic event', 4):
            # st.subheader('')
            path_data_1 = '/content/gdrive/My Drive/materials/match_teamfight_count_mean_chatnum_toxic.csv'
            df_data_1 = load_data(path_data_1)
            # df_data_1[['first_blood_time', 'toxic_words_count']]
            df_data_1[['first_blood_time', 'toxic_words_count']].sort_values("first_blood_time", inplace=False)
            path_data_2 = '/content/gdrive/My Drive/materials/players.csv'
            df_data_2 = load_data(path_data_2)
            result = pd.merge(df_data_1, df_data_2, how='outer', on=['match_id'])
            listbin2s = [i * 5 for i in range(13)]
            result['kills'] = pd.cut(result['kills'], listbin2s, right=True, labels=None, retbins=False, precision=3,
                                     include_lowest=False)
            thirdy = list(
                result[['kills', 'toxic_words_count']].sort_values("kills", inplace=False).groupby('kills').mean()[
                    'toxic_words_count'])

            plt.figure(4)
            l1 = plt.plot(listbin2s[1:], thirdy)

            plt.title('Relation between kills and toxic event')
            plt.xlabel('kills')
            plt.ylabel('toxic_word_count')

            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that the relation between toxic word and kill event.
                """)

        if st.checkbox('5. Player skills level distribution (sigma)', 5):
            path_data = '/content/gdrive/My Drive/materials/player_ratings.csv'
            df_data = load_data(path_data)
            trueskill_sigma = df_data["trueskill_sigma"].values
            total_matches = df_data["total_matches"].values

            plt.figure(5)

            plt.scatter(total_matches[0:-1],trueskill_sigma[0:-1])
            plt.xlabel('total matches')
            plt.ylabel('trueskill sigma')
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that the player's true skill will tend to be stable while the total matches increases.
                """)

    elif choice == 'Dataset Analysis 2':
        if st.checkbox('1. Top 30 Most Frequent Bad Words in the Game', 1):
            # st.subheader('')
            path_data = '/content/gdrive/My Drive/materials/Output.csv'
            chat_df= load_data(path_data)
            # chat_df.columns = ['match_id', 'key', 'slot', 'time', 'unit']
            chat_string = ' '.join(chat for chat in chat_df['key'])
            # lower case and remove extra white spaces
            chat_string = chat_string.lower()
            chat_string = re.sub(r'\s+', ' ', chat_string)

            # create spacy document
            len(chat_string)
            nlp.max_length = 16700000
            nlp.Defaults.stop_words.add('u')
            chat_doc = nlp(chat_string)

            toxic_words_lexicon = load_lexcion()

            # match lexicon with document
            matcher = PhraseMatcher(nlp.vocab)
            terms_list = toxic_words_lexicon
            patterns = [nlp.make_doc(text) for text in terms_list]
            matcher.add('phrase_matcher', None, *patterns)
            character_matches = matcher(chat_doc)

            c = []
            for match_id, start, end in character_matches:
                rule_id = nlp.vocab.strings[match_id]
                span = chat_doc[start:end]
                c.append(span.text)
            # print(c)

            # Calculate top 30 bad words frequency
            batwords_word_freq = Counter(c)
            top30badwords = batwords_word_freq.most_common(30)

            freq_df = pd.DataFrame.from_dict(batwords_word_freq, orient='index').reset_index()
            freq_df.columns = ["word", "freq"]
            freq_df.sort_values(by="freq", ascending=False).head(30)


            plt.figure(1)
            fig, ax = plt.subplots(figsize=(20, 10))
            sns.barplot(data=freq_df.sort_values(by="freq", ascending=False).head(30),
                        y="word",
                        x="freq",
                        color='#7bbcd5')
            plt.ylabel("Word")
            plt.xlabel("Frequency")
            plt.title("Top 30 Most Frequent Bad Words in the Game")
            sns.despine()

            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that Top 30 Most Frequent Bad Words in the Game.
                """)

        if st.checkbox('2. First blood time in the game', 2):

            path_data = '/content/gdrive/My Drive/materials/match.csv'
            df_data= load_data(path_data)

            first_blood_temp = df_data['first_blood_time']
            first_blood = []
            for i in first_blood_temp:
                if i != 0:
                    first_blood.append(i)
            first_blood_mean = sum(first_blood) / len(first_blood)
            first_blood_max = max(first_blood)
            first_blood_min = min(first_blood)

            plt.figure(2)

            plt.boxplot(first_blood, showfliers=False)
            plt.ylabel('Time')
            plt.xticks(labels=['First Blood'], ticks=[1])

            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates that the first blood time in the game.
                """)


        if st.checkbox('3. Relation between kills and toxic event', 3):
            # st.subheader('')
            path_data_1 = '/content/gdrive/My Drive/materials/match_teamfight_count_mean_chatnum_toxic.csv'
            df_data_1 = load_data(path_data_1)
            # df_data_1[['first_blood_time', 'toxic_words_count']]
            df_data_1[['first_blood_time', 'toxic_words_count']].sort_values("first_blood_time", inplace=False)
            path_data_2 = '/content/gdrive/My Drive/materials/players.csv'
            df_data_2 = load_data(path_data_2)
            result = pd.merge(df_data_1, df_data_2, how='outer', on=['match_id'])

            listbins = [i*5 for i in range(10)]
            listbin2s = [i * 5 for i in range(13)]
          
            result['deaths'] = pd.cut(result['deaths'],listbins,right=True,labels=None,retbins=False,precision=3,include_lowest=False)
            result['kills'] = pd.cut(result['kills'],listbin2s,right=True,labels=None,retbins=False,precision=3,include_lowest=False)

            resulty = list(result[['deaths','toxic_words_count']].sort_values("deaths",inplace=False).groupby('deaths').mean()['toxic_words_count'])
            thirdy = list(result[['kills','toxic_words_count']].sort_values("kills",inplace=False).groupby('kills').mean()['toxic_words_count'])

            plt.figure(3)
            l1=plt.plot(listbins[1:],resulty)
            l2=plt.plot(listbin2s[1:10],thirdy[:9],label = 'kills')

            plt.title('Relation between deaths/kills and toxic event')
            plt.xlabel('deaths/kills')
            plt.ylabel('toxic_word_count')
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates the relation between deaths/kills and toxic event.
                """)


        if st.checkbox('4. Teamfights start points', 4):


            x,y = load_teamfights()

            plt.figure(4)
            plt.plot(x,y)
            plt.ylabel("Count of start points")
            plt.xlabel("Time series(second)")
            plt.title("Scatter plots of teamfights start points")
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates start points of teamfights.
                """)

        if st.checkbox('5. Teamfights end points', 5):


            x1,y1 = load_teamfights_end()

            plt.figure(5)
            plt.plot(x1,y1)
            plt.ylabel("Count of end points")
            plt.xlabel("Time series(second)")
            plt.title("Scatter plots of teamfights end points")
            st.pyplot()

            with st.beta_expander("Check analysis"):
                st.write("""
                    This figure indicates end points of teamfights.
                """)
    # st.write(user_input.dtypes)

@st.cache
def load_teamfights_end():
    path_data = '/content/gdrive/My Drive/materials/teamfights.csv'
    df_data = load_data(path_data)

    teamfight_end = df_data['end']
    i1=100
    x1=[]
    y1=[]
    while i1<16000:
      x1.append(i1);
      temp1=0
      for a in teamfight_end:
        if a <=i1 and a >=i1-10:
          temp1+=1
      y1.append(temp1)
      i1+=100

    return x1, y1


@st.cache
def load_teamfights():
    path_data = '/content/gdrive/My Drive/materials/teamfights.csv'
    df_data = load_data(path_data)

    teamfight_start = df_data['start']
    i1=100
    x=[]
    y=[]
    while i1<16000:
      x.append(i1);
      temp1=0
      for a in teamfight_start:
        if a <=i1 and a >=i1-10:
          temp1+=1
      y.append(temp1)
      i1+=100
    
    return x,y 


@st.cache
def load_lexcion():
    toxic_words = []

    # 需要自己重新上传list_google_lexcon.txt
    for line in open("/content/gdrive/My Drive/materials/list_google_lexicon.txt"):
        toxic_words.append(line[:-1])

    for line in open("/content/gdrive/My Drive/materials/profanity_wordlist.txt"):
        toxic_words.append(line[:-1])

    # 这部分可以自己手动添加新的list 扩充lexicon
    toxic_words.extend(
        ['noob', 'noobs', 'retard', 'retards', 'tard', 'tards', 'idiot', 'idiots', 'stupid', 'moron', 'morons', 'fu',
         'nap'])

    toxic_words_new = []
    for i in toxic_words:
        if i not in toxic_words_new:
            toxic_words_new.append(i)

    return toxic_words_new

@st.cache
def load_data(path):
    df = pd.read_csv(path, header=0)
    return df


class Config(object):
    """配置参数"""

    def __init__(self):
        self.model_name = 'bert'  # 测试集
        self.class_list = [x.strip() for x in open(
            '/content/gdrive/My Drive/materials/labels_phrase_opi.txt').readlines()]  # 类别名单
        self.save_path = '/content/gdrive/My Drive/materials/saved_dict_opi_ordered_chen_new'
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 设备

        # self.require_improvement = 1000  # 若超过1000batch效果还没提升，则提前结束训练

        self.require_improvement = 100000 

        self.num_classes = len(self.class_list)
        # self.num_classes = 2                         # 类别数
        self.num_epochs = 5  # epoch数

        # self.batch_size = 64           
        self.batch_size = 1  # mini-batch大小

        self.pad_size = 32  # 每句话处理成的长度(短填长切)
        self.learning_rate = 5e-5
        # 改成下载的预训练模型路径
        # self.bert_path = ('/content/gdrive/My Drive/materials/bert-base-uncased')

        self.bert_model_choose = "bert-base-uncased"
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # self.bert_model_choose = "bert-base-multilingual-uncased"
        # self.tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

        self.hidden_size = 768


config = Config()


class DatasetIterater(object):
    def __init__(self, batches, batch_size, device):
        self.batch_size = batch_size
        self.batches = batches
        self.n_batches = len(batches) // batch_size
        self.residue = False  # 记录batch数量是否为整数
        if len(batches) % self.n_batches != 0:
            self.residue = True
        self.index = 0
        self.device = device

    def _to_tensor(self, datas):
        x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
        y = torch.LongTensor([_[1] for _ in datas]).to(self.device)

        # pad前的长度(超过pad_size的设为pad_size)
        seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
        mask = torch.LongTensor([_[3] for _ in datas]).to(self.device)


        # print("x is ", x)
        # print("y is ", y)
        # print("seq len is ", seq_len)
        # print("mask is ", mask)

        return (x, seq_len, mask), y

    def __next__(self):
        if self.residue and self.index == self.n_batches:
            batches = self.batches[self.index * self.batch_size: len(self.batches)]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

        elif self.index >= self.n_batches:
            self.index = 0
            raise StopIteration
        else:
            batches = self.batches[self.index * self.batch_size: (self.index + 1) * self.batch_size]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

    def __iter__(self):
        return self

    def __len__(self):
        if self.residue:
            return self.n_batches + 1
        else:
            return self.n_batches


class Model(nn.Module):

    def __init__(self, config):
        super(Model, self).__init__()
        # self.bert = BertModel.from_pretrained(config.bert_path)
        self.bert = BertModel.from_pretrained(config.bert_model_choose)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(config.hidden_size, config.num_classes)

    def forward(self, x):
        context = x[0]
        print("context is ", context)
        mask = x[2]
        print("mask is ", mask)
        _, pooled = self.bert(context, attention_mask=mask)
        out = self.fc(pooled)

        return out


def load_dataset(test_data, pad_size=32):
    contents = []
    token = config.tokenizer.tokenize(test_data)
    token = [CLS] + token
    seq_len = len(token)
    mask = []
    token_ids = config.tokenizer.convert_tokens_to_ids(token)
    label = "0"

    if pad_size:
        if len(token) < pad_size:
            mask = [1] * len(token_ids) + [0] * (pad_size - len(token))
            token_ids += ([0] * (pad_size - len(token)))
        else:
            mask = [1] * pad_size
            token_ids = token_ids[:pad_size]
            seq_len = pad_size
    contents.append((token_ids, int(label), seq_len, mask))
    return contents


def build_iterator(dataset, config):
    iter = DatasetIterater(dataset, config.batch_size, config.device)
    return iter



def evaluate(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    print('_____')
    with torch.no_grad():
        for texts, labels in data_iter:
            outputs = model(texts)
            print("ssfsf si", outputs)
            loss = F.cross_entropy(outputs, labels)
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = torch.max(outputs.data, 1)[1].cpu().numpy()

            print("lal is ", predic)

            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)

            print("lal1 is ", predict_all)

    return predict_all

def acp_use_input():
    st.write("Enter your text: ")
    text_input = st.text_input(" ", value='', key=None)
    # df_input = pd.DataFrame(np.array([text_input]).reshape(1, -1))
        #
        # return df_input
    return text_input


if __name__ == '__main__':
    main()



Overwriting web.py


In [8]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://2a5232c07eda.ngrok.io


In [ ]:
!streamlit run web.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.232.135.251:8501

2020-12-04 13:29:42.930 Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
2020-12-04 13:29:43.304269: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-12-04 13:29:46.613 Lock 140263853967288 acquired on /root/.cache/torch/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock
Downloading: 100% 433/433 [00:00<00:00, 415kB/s]
2020-12-04 13:29:46.941 Lock 140263853967288 released on /root/.cache/torch/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2